<a href="https://colab.research.google.com/github/sunami20/Data-Science-2025/blob/main/assignment_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, classification_report



In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/Housing.csv')

print(df.head())

      price  area  bedrooms  bathrooms  stories mainroad guestroom basement  \
0  13300000  7420         4          2        3      yes        no       no   
1  12250000  8960         4          4        4      yes        no       no   
2  12250000  9960         3          2        2      yes        no      yes   
3  12215000  7500         4          2        2      yes        no      yes   
4  11410000  7420         4          1        2      yes       yes      yes   

  hotwaterheating airconditioning  parking prefarea furnishingstatus  
0              no             yes        2      yes        furnished  
1              no             yes        3       no        furnished  
2              no              no        2      yes   semi-furnished  
3              no             yes        3      yes        furnished  
4              no             yes        2       no        furnished  


In [ ]:
#Baasic exploration

print('Dataset Shape: ', df.shape)

Dataset Shape:  (545, 13)


In [ ]:
# information show
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 545 entries, 0 to 544
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   price             545 non-null    int64 
 1   area              545 non-null    int64 
 2   bedrooms          545 non-null    int64 
 3   bathrooms         545 non-null    int64 
 4   stories           545 non-null    int64 
 5   mainroad          545 non-null    object
 6   guestroom         545 non-null    object
 7   basement          545 non-null    object
 8   hotwaterheating   545 non-null    object
 9   airconditioning   545 non-null    object
 10  parking           545 non-null    int64 
 11  prefarea          545 non-null    object
 12  furnishingstatus  545 non-null    object
dtypes: int64(6), object(7)
memory usage: 55.5+ KB
None


In [ ]:
# Basic statistics
print(df.describe())

              price          area    bedrooms   bathrooms     stories  \
count  5.450000e+02    545.000000  545.000000  545.000000  545.000000   
mean   4.766729e+06   5150.541284    2.965138    1.286239    1.805505   
std    1.870440e+06   2170.141023    0.738064    0.502470    0.867492   
min    1.750000e+06   1650.000000    1.000000    1.000000    1.000000   
25%    3.430000e+06   3600.000000    2.000000    1.000000    1.000000   
50%    4.340000e+06   4600.000000    3.000000    1.000000    2.000000   
75%    5.740000e+06   6360.000000    3.000000    2.000000    2.000000   
max    1.330000e+07  16200.000000    6.000000    4.000000    4.000000   

          parking  
count  545.000000  
mean     0.693578  
std      0.861586  
min      0.000000  
25%      0.000000  
50%      0.000000  
75%      1.000000  
max      3.000000  


In [ ]:
# Missing Values Check
print(df.isnull().sum())

price               0
area                0
bedrooms            0
bathrooms           0
stories             0
mainroad            0
guestroom           0
basement            0
hotwaterheating     0
airconditioning     0
parking             0
prefarea            0
furnishingstatus    0
dtype: int64


In [ ]:
# Target variable distributaion

print(df['furnishingstatus'].value_counts())

furnishingstatus
semi-furnished    227
unfurnished       178
furnished         140
Name: count, dtype: int64


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Copy df to avoid modifying original
df_encoded = df.copy()

# Identify categorical columns for X (excluding the target 'furnishingstatus')
categorical_cols_for_X = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']

# Apply one-hot encoding for these categorical columns in X
# drop_first=True avoids multicollinearity
df_encoded = pd.get_dummies(df_encoded, columns=categorical_cols_for_X, drop_first=True, dtype=int)

# Separate input (X) and output (Y)
X = df_encoded.drop('furnishingstatus', axis=1)
Y = df_encoded['furnishingstatus']

# Encode the target variable Y
label_encoder_y = LabelEncoder()
Y = label_encoder_y.fit_transform(Y)

print('X has all numerical input features (after encoding)')
print('Y has encoded furnishingstatus (0, 1, or 2)')

# Split: 80 % for trainning, 20% for testing
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y,
    test_size= 0.2,
    random_state =20
)

print(f'We will train with: {len(X_train)} samples')
print(f'We will test with: {len(X_test)} samples')

X has all patient informationn (8 comlumns)
y has Housing yes/no (1 column)
 we will train with: 436 owner
 we will test with: 109 owner


# **why scale **

In [ ]:
# Problem: Some numbers are big, some are small

print("Before scalling - different ranges:")
print(f"Age: smallest={X_train['bathrooms'].min()}, biggest={X_train['bathrooms'].max()}")
print(f'Price: smallest={X_train['bedrooms'].min()}, biggest={X_train['bedrooms'].max()}')

Before scalling - different ranges:
Age: smallest=1, biggest=4
Price: smallest=1, biggest=6


# **#Standdard Scaler**

In [ ]:
# creat scaler
scaler = StandardScaler()

# Learn from training data and scale it
X_train_scaled = scaler.fit_transform(X_train)

# Scale test data (just transform, don't fit)
X_test_scaled = scaler.transform(X_test)

print("\n After Scaling")
print(" All features now have similar range!")

# Adjusted print statement for new column names after get_dummies
# We'll check 'basement_yes' column which was originally 'basement'
if 'basement_yes' in X.columns:
    basement_col_idx = X.columns.get_loc('basement_yes')
    print(f"Example - First housing basement before scaling: {X_train.iloc[0][basement_col_idx]}")
    print(f"Example - First housing basement after scaling: {X_train_scaled[0][basement_col_idx]}")
else:
    print("Could not find 'basement_yes' column for example print.")


 After Scaling
 All features now have similar range!
Could not find 'basement_yes' column for example print.


# **MinMaxScaler (Min-Max normalized)**

In [ ]:
# Craete MinMaxscaler
minmax_scaler = MinMaxScaler()

# Learn from training data and scale it
X_train_minmax = minmax_scaler.fit_transform(X_train)

# Scale test data
X_test_minmax =minmax_scaler.transform(X_test)

print("\n After MinMaxScaler:")
print(f"Example - First housing age before scaling: {X_train.iloc[0][7]}")
print(f"Example - First housing age after scaling: {X_train_minmax[0][7]}")



 After MinMaxScaler:
Example - First housing age before scaling: nan
Example - First housing age after scaling: nan


# **Encoding Categorical Data**

In [ ]:
# Let's create a sample dataset with text data
sample_data = pd.DataFrame({
    'age': [23, 26, 35, 40],
    'buyer': ['male', 'female', 'male', 'female'],
    'agree': ['yes', 'no', 'yes', 'no'],
    'problem': [1,0,1,0]
})

print(sample_data)

   age   buyer agree  problem
0   23    male   yes        1
1   26  female    no        0
2   35    male   yes        1
3   40  female    no        0


# **Label Encoding (for Binary/Two categories)**

In [ ]:
from sklearn.preprocessing import LabelEncoder

# For yes? no or male /female
label_encoder = LabelEncoder()

# convert agree colum
sample_data['agree_encoded'] = label_encoder.fit_transform(sample_data['agree'])
print("\n After Label Encoding 'agree' :")
print(sample_data[['agree', 'agree_encoded']])

# convert 'buyer' column (corrected from 'gender')
sample_data['gender_encoded'] = label_encoder.fit_transform(sample_data['buyer'])
print("\nAfter Label Encoding 'buyer':")
print(sample_data[['buyer', 'gender_encoded']])


 After Label Encoding 'agree' :
  agree  agree_encoded
0   yes              1
1    no              0
2   yes              1
3    no              0

After Label Encoding 'buyer':
    buyer  gender_encoded
0    male               1
1  female               0
2    male               1
3  female               0


# **One Hot Encoding**

In [ ]:
#let;s add a column with multiple categories
sample_data['city'] =['Delhi', 'Mumbai', 'Delhi', 'Kolkata']

# One-Hot Encoding using pandas
city_encoded = pd.get_dummies(sample_data['city'], prefix='city')
print("n One-Hot Encoding using for city" )
print(city_encoded)

# Creates separete columns for each city

# Add to original data
sample_data = pd.concat([sample_data, city_encoded], axis=1)
print('\n Final data with all encoding: ')
print(sample_data)

n One-Hot Encoding using for city
   city_Delhi  city_Kolkata  city_Mumbai
0        True         False        False
1       False         False         True
2        True         False        False
3       False          True        False

 Final data with all encoding: 
   age   buyer agree  problem  agree_encoded  gender_encoded     city  \
0   23    male   yes        1              1               1    Delhi   
1   26  female    no        0              0               0   Mumbai   
2   35    male   yes        1              1               1    Delhi   
3   40  female    no        0              0               0  Kolkata   

   city_Delhi  city_Kolkata  city_Mumbai  
0        True         False        False  
1       False         False         True  
2        True         False        False  
3       False          True        False  


# **Train a Simple Model**

In [ ]:
from sklearn.linear_model import LogisticRegression

# Create model
model = LogisticRegression(max_iter=1000)

# Train the model
model.fit(X_train_scaled, Y_train)
print(" Model trained!")

# check accuracy on training data
train_score = model.score(X_train_scaled, Y_train)
print(f"Tranining accuracy: {train_score:.1%}")

# Check accuracy on test data
test_score = model.score(X_test_scaled,Y_test)
print(f"Test accuracy: {test_score:.1%}")


ValueError: Input X contains NaN.
LogisticRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

# **Cross - validation**

In [ ]:
from sklearn.model_selection import cross_val_score

# Do 5-fold cross-vlaidation
scores = cross_val_score(
    model,
    X_train_scaled,
    Y_train,
    cv=5
)

print("5 mini-test scores")
for i in range(5):
  print(f" test {i+1}: {scores[i]:.1%}")
  print(f"n Average: {scores.mean():1%}")
  print(f"This means our model is {scores.mean():1%} accurate")

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.12/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py", line 1222, in fit
    X, y = validate_data(
           ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py", line 2961, in validate_data
    X, y = check_X_y(X, y, **check